In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

def transform(machine,name):
    time_stamps = []
    for elt in machine.columns :
        if str.startswith(elt,'TimeStep') :
            time_stamps.append(elt)
    machine = pd.melt(machine,id_vars=['Index','House_id'],value_vars= time_stamps, var_name='time',value_name = name)

    machine = pd.DataFrame(machine)
    times = []
    for elt in machine['time']:
        times.append(int(elt[9:]))
    machine['time'] = times
    machine = machine.sort_values(['Index','time'])
    return machine


In [2]:
consumption =pd.read_csv("InputTrain.csv")
washing_machine = pd.read_csv("StepTwo_LabelTrain_WashingMachine.csv")
dishwasher = pd.read_csv("StepTwo_LabelTrain_Dishwasher.csv")
tumble_dryer = pd.read_csv("StepTwo_LabelTrain_TumbleDryer.csv")
microwave = pd.read_csv("StepTwo_LabelTrain_Microwave.csv")
kettle = pd.read_csv("StepTwo_LabelTrain_Kettle.csv")


washing_machine = transform(washing_machine,'Washing Machine')
dishwasher = transform(dishwasher,'Dishwasher')
tumble_dryer = transform(tumble_dryer,'Tumble Dryer')
microwave = transform(microwave,'Microwave')
kettle = transform(kettle,'Kettle')

machines = [washing_machine,dishwasher,tumble_dryer,microwave,kettle]

consumption = transform(consumption,'watt')
display(consumption)



,Index,House_id,time,watt
0,0,1,0,180.0
10421,0,1,1,180.0
20842,0,1,2,180.0
31263,0,1,3,181.0
41684,0,1,4,180.0
...,...,...,...,...
22467675,10420,16,2155,366.5
22478096,10420,16,2156,376.0
22488517,10420,16,2157,367.0
22498938,10420,16,2158,353.0


In [3]:

df_train = pd.concat([consumption,washing_machine['Washing Machine'],dishwasher['Dishwasher'],tumble_dryer['Tumble Dryer'],microwave['Microwave'],kettle['Kettle']],names=['Index', 'House_id','time','watt','Washing Machine', 'Dishwasher', 'Tumble Dryer', 'Microwave', 'Kettle', 'watt'],axis=1,join="inner")
display(df_train)

,Index,House_id,time,watt,Washing Machine,Dishwasher,Tumble Dryer,Microwave,Kettle
0,0,1,0,180.0,0.0,0.0,0.0,0.0,0.0
10421,0,1,1,180.0,0.0,0.0,0.0,0.0,0.0
20842,0,1,2,180.0,0.0,0.0,0.0,0.0,0.0
31263,0,1,3,181.0,0.0,0.0,0.0,0.0,0.0
41684,0,1,4,180.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
22467675,10420,16,2155,366.5,0.0,0.0,0.0,0.0,0.0
22478096,10420,16,2156,376.0,0.0,0.0,0.0,0.0,0.0
22488517,10420,16,2157,367.0,0.0,0.0,0.0,0.0,0.0
22498938,10420,16,2158,353.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Paramètres du modèle
batch_size = 32
seq_len = 2160
n_features = 1
n_targets = 5

# Création du générateur de données d'entraînement
def generate_data(df, batch_size):
    num_curves = len(df['Index'].unique())
    num_batches_per_epoch = num_curves // batch_size

    while True:
        # Mélange aléatoire des courbes
        shuffled_curves = np.random.permutation(df['Index'].unique())

        # Pour chaque batch
        for i in range(num_batches_per_epoch):
            # Sélection aléatoire de batch_size courbes
            batch_curves = shuffled_curves[i*batch_size:(i+1)*batch_size]

            # Initialisation des tenseurs d'entrée et de sortie du batch
            X_batch = np.zeros((batch_size, seq_len, n_features))
            y_batch = np.zeros((batch_size, seq_len, n_targets))

            # Remplissage des tenseurs d'entrée et de sortie du batch
            for j, curve in enumerate(batch_curves):
                curve_data = df[df['Index'] == curve]
                X_batch[j,:,:] = curve_data[['watt']].values
                y_batch[j,:,:] = curve_data[['Washing Machine', 'Dishwasher', 'Tumble Dryer', 'Microwave', 'Kettle']].values

            yield (X_batch, y_batch)

    


In [ ]:
# Création du modèle
model = Sequential()
model.add(LSTM(units=128, input_shape=(seq_len, n_features), return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(64, activation="sigmoid"))
model.add(LSTM(units=128, input_shape=(seq_len, n_features), return_sequences=True))
model.add(Dense(n_targets, activation="sigmoid"))

# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy')

# Entraînement du modèle
train_generator = generate_data(df_train, batch_size=batch_size)
for i in range(5):
    model.save('modeltest3.h5')
    model.fit(train_generator, steps_per_epoch=500, epochs=1)
model.save('modeltest3.h5')




In [8]:
model = tf.keras.models.load_model('modeltest3.h5')
train_generator = generate_data(df_train, batch_size=batch_size)
for i in range(100):
    model.fit(train_generator, steps_per_epoch=200, epochs=1)
    model.save('modeltest3.h5')


  1/200 [..............................] - ETA: 10:20 - loss: 0.0268

KeyboardInterrupt: 

In [9]:
model = tf.keras.models.load_model('modeltest3.h5')
# Charger les données de test dans une DataFrame Pandas
df_test = pd.read_csv("InputTest.csv")
df_test = transform(df_test,"watt")


# Génération des données de test
def generate_test_data(df):
    for _, curve_data in df.groupby('Index'):
        X = np.array(curve_data[['watt']].values).reshape(1, -1, 1)
        yield X

# Prédiction des états des appareils pour chaque courbe de consommation
y_pred = []
for X_test in generate_test_data(df_test):
    y_test_pred = model.predict(X_test)
    y_pred.append(y_test_pred[0,:,:])

# Création d'un DataFrame pour les résultats
result_df = pd.DataFrame(data=np.array(y_pred).reshape(-1, 5), columns=['Washing Machine', 'Dishwasher', 'Tumble Dryer', 'Microwave', 'Kettle'])


result_df = result_df.round(4)
display(result_df)
# Exportation des résultats au format CSV
result_df.to_csv('results_multi.csv', index=True,index_label="Index")

1/1 [==============================] - 0s 87ms/step


,Washing Machine,Dishwasher,Tumble Dryer,Microwave,Kettle
0,0.0362,0.0355,0.0063,0.0024,0.0044
1,0.0395,0.0131,0.0024,0.0002,0.0001
2,0.0268,0.0105,0.0013,0.0000,0.0000
3,0.0268,0.0113,0.0013,0.0000,0.0000
4,0.0188,0.0109,0.0016,0.0000,0.0000
...,...,...,...,...,...
5374075,0.0002,0.9962,0.0000,0.0000,0.0000
5374076,0.0002,0.9962,0.0000,0.0000,0.0000
5374077,0.0002,0.9962,0.0000,0.0000,0.0000
5374078,0.0002,0.9962,0.0000,0.0000,0.0000
